In [1]:
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei']
from nltk import word_tokenize     
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
df=pd.read_csv("/Users/cindychen/Desktop/consulting group project/Job_Data.csv")

#remove punctuation
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
df.head()

,position,company,location,about
0,Applications Architect Crawley Based,Chubb,Crawley England United Kingdom,The Application Architect will be responsible ...
1,Modernization Application Developer IBM CIC,IBM,Hursley England United Kingdom,Introduction As an Application Developer you...
2,Open Source Architect,IBM,Manchester England United Kingdom,Introduction This is a new and exciting role ...
3,Data Engineer IBM CIC,IBM,London England United Kingdom,Introduction We are looking for a Data Engine...
4,IBM TSS Pre sales Consultant,IBM,Warwick England United Kingdom,Introduction At IBM work is more than a job ...


In [3]:
# Convertng upper case to lower case
df['about']=df['about'].str.lower()

df['about'].head()

0    the application architect will be responsible ...
1    introduction  as an application developer  you...
2    introduction  this is a new and exciting role ...
3    introduction  we are looking for a data engine...
4    introduction  at ibm  work is more than a job ...
Name: about, dtype: object

In [4]:
#creat a corpus 
corpus = []
#eg:['mine collapsed german states to spend around','a lock that has been built to','microsoft corp said on wednesday']
for i in range(len(df)):
    corpus.append(df['about'][i])
print(corpus)

['the application architect will be responsible for the enablement of chubb s platforms  reference architecture models and capabilities  their adoption for an effective design and development of business solutions and their integration into targeted business ecosystems   responsibilities  develop flexible  adaptable  modular and reusable business solution architecture designs in collaboration with product delivery and operation support teams  develop solutions and roadmaps on applications transformation and modernisation  support evolution of architecture blueprints   reference models and facilitate their enablement and adoption within the enterprise  implement appropriate solution architecture governance processes and tools while maintaining a key focus on design patterns and technology standards  maintain a repository of architecture assets  platforms  services  apis  and promote effective reuse across the enterprise  act as an enabler of agile and devops practices in collaboration w

In [5]:
# implement BAG OF WORDS
def norm_data(data, norm):  
    # This function is used for matrix normalization. 
    # The input data and output data_norm can be a two-dimensional matrix or a csr matrix
    """
    norm = 'l1', 'l2' or 'l0' or 'l2+l0'...
        'l0': normalize the data matrix as the sum of all entries=1
    """
    data_norm = data
    norms = norm.split("+")  # the sequence of norm
    for norm in norms:
        if norm in ['l1', 'l2']:
            data_norm = normalize(data_norm, norm, axis=1, copy=True)  # donot change the original input data
        if norm == "l0":
            _sum = data.sum()
            data_norm = data_norm / _sum
    return data_norm

In [6]:
all_key_words=[]
def load_tfidf(corpu):  
    # This is a list, each element represents a headline
    tfidf_vect = TfidfVectorizer()
    tfidf__matrix = tfidf_vect.fit_transform(corpu)
    feature_names = tfidf_vect.get_feature_names()  
    # Get all the unique words in the corpus，and align them with the final matrix order
    vocabulary = tfidf_vect.vocabulary_  
    # It is a set, the length is the number of data, get how many times each word appears
    # print(vocabulary)
    all_key_words.append(vocabulary)

    tfidf__matrix_toarray = tfidf__matrix.toarray()  
# Two-dimensional array which get all get the vector corresponding to each sentence, 
# The order of the numbers in the vector is in the order of the id (feature_names) of the words

    return tfidf__matrix_toarray
news_title_matrix = norm_data(data=load_tfidf(corpu=corpus), norm="l2") # Two-dimensional array
all_key_words=all_key_words[0]
print(all_key_words)

{'the': 1450, 'application': 86, 'architect': 98, 'will': 1588, 'be': 158, 'responsible': 1247, 'for': 601, 'enablement': 487, 'of': 973, 'chubb': 241, 'platforms': 1072, 'reference': 1199, 'architecture': 101, 'models': 916, 'and': 72, 'capabilities': 215, 'their': 1451, 'adoption': 34, 'an': 66, 'effective': 466, 'design': 408, 'development': 420, 'business': 204, 'solutions': 1353, 'integration': 749, 'into': 760, 'targeted': 1428, 'ecosystems': 463, 'responsibilities': 1245, 'develop': 416, 'flexible': 592, 'adaptable': 26, 'modular': 920, 'reusable': 1254, 'solution': 1351, 'designs': 411, 'in': 718, 'collaboration': 267, 'with': 1590, 'product': 1127, 'delivery': 396, 'operation': 994, 'support': 1406, 'teams': 1433, 'roadmaps': 1267, 'on': 981, 'applications': 87, 'transformation': 1488, 'modernisation': 918, 'evolution': 543, 'blueprints': 180, 'facilitate': 575, 'within': 1591, 'enterprise': 507, 'implement': 711, 'appropriate': 94, 'governance': 653, 'processes': 1124, 'tools

In [7]:
# Expert to csv

a=[]
b=[]
# a=all_key_words.keys()
# b=all_key_words.values()

for name,score in all_key_words.items():
    a.append(name)
    b.append(score)

out_put=pd.DataFrame()
out_put['Keywords']=list(a)
out_put['Count']=list(b)

print(out_put)
out_put.to_csv("/Users/cindychen/Desktop/consulting group project/keywords.csv", index=False)

         Keywords  Count
0             the   1450
1     application     86
2       architect     98
3            will   1588
4              be    158
...           ...    ...
1608      undergo   1509
1609     criteria    359
1610         need    947
1611          ilr    706
1612        worth   1600

[1613 rows x 2 columns]


In [8]:
# Print the top 100 keywords
key_words_100=sorted(zip( all_key_words.keys(),all_key_words.values()),reverse=True)[0:100]
print(key_words_100)

[('zone', 1612), ('zero', 1611), ('zeppelin', 1610), ('zachman', 1609), ('yourself', 1608), ('your', 1607), ('you', 1606), ('years', 1605), ('year', 1604), ('written', 1603), ('write', 1602), ('would', 1601), ('worth', 1600), ('worldwide', 1599), ('world', 1598), ('workshops', 1597), ('workplace', 1596), ('working', 1595), ('worked', 1594), ('work', 1593), ('without', 1592), ('within', 1591), ('with', 1590), ('willing', 1589), ('will', 1588), ('wider', 1587), ('wide', 1586), ('why', 1585), ('wholly', 1584), ('whole', 1583), ('who', 1582), ('whilst', 1581), ('while', 1580), ('which', 1579), ('whether', 1578), ('where', 1577), ('when', 1576), ('whatever', 1575), ('what', 1574), ('were', 1573), ('well', 1572), ('welcome', 1571), ('websphere', 1570), ('websocket', 1569), ('webex', 1568), ('webdriver', 1567), ('web', 1566), ('we', 1565), ('ways', 1564), ('way', 1563), ('watson', 1562), ('waterfall', 1561), ('wants', 1560), ('want', 1559), ('vue', 1558), ('voice', 1557), ('vital', 1556), ('v

In [9]:
# Keywords which more than 50 times
nes_key_words={name: score for name,score in all_key_words.items() if score > 50}
nes_key_words = sorted(zip( nes_key_words.keys(),nes_key_words.values()),reverse=True)
print(nes_key_words)

[('zone', 1612), ('zero', 1611), ('zeppelin', 1610), ('zachman', 1609), ('yourself', 1608), ('your', 1607), ('you', 1606), ('years', 1605), ('year', 1604), ('written', 1603), ('write', 1602), ('would', 1601), ('worth', 1600), ('worldwide', 1599), ('world', 1598), ('workshops', 1597), ('workplace', 1596), ('working', 1595), ('worked', 1594), ('work', 1593), ('without', 1592), ('within', 1591), ('with', 1590), ('willing', 1589), ('will', 1588), ('wider', 1587), ('wide', 1586), ('why', 1585), ('wholly', 1584), ('whole', 1583), ('who', 1582), ('whilst', 1581), ('while', 1580), ('which', 1579), ('whether', 1578), ('where', 1577), ('when', 1576), ('whatever', 1575), ('what', 1574), ('were', 1573), ('well', 1572), ('welcome', 1571), ('websphere', 1570), ('websocket', 1569), ('webex', 1568), ('webdriver', 1567), ('web', 1566), ('we', 1565), ('ways', 1564), ('way', 1563), ('watson', 1562), ('waterfall', 1561), ('wants', 1560), ('want', 1559), ('vue', 1558), ('voice', 1557), ('vital', 1556), ('v